In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
import yaml
import numpy as np
import pandas as pd
from func.utils import get_categorical_features
from func.ml_utils import Classifier
from func.BigQuery import BigQuery

In [30]:
COLUMN_ID = 'TransactionID'
COLUMN_TARGET = 'isFraud'
COLUMNS_IGNORE = [COLUMN_ID, COLUMN_TARGET]
COLUMNS_CATEGORY = get_categorical_features(dataset, COLUMNS_IGNORE)

dataset = pd.read_csv('../input/train_transaction.csv')
use_cols = [col for col in dataset.columns if col not in COLUMNS_IGNORE+COLUMNS_CATEGORY]
Y = dataset[COLUMN_TARGET]
train_df = dataset[use_cols]
train_df.head()

,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,dist2,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,86400,68.5,13926,NaN,150.0,142.0,315.0,87.0,19.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,86401,29.0,2755,404.0,150.0,102.0,325.0,87.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,86469,59.0,4663,490.0,150.0,166.0,330.0,87.0,287.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,86499,50.0,18132,567.0,150.0,117.0,476.0,87.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,86506,50.0,4497,514.0,150.0,102.0,420.0,87.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
from sklearn.model_selection import StratifiedKFold

n_splits = 5
kfold = StratifiedKFold(n_splits=n_splits)
kfold = list(kfold.split(train_df, Y))
params = {}
model_type = 'lgb'
score_list = []
feim_list = []
y_pred = np.zeros(len(train_df))
test_preds = []
use_cols = COLUMNS_TOP100

for n_fold, (trn_idx, val_idx) in enumerate(kfold):
    x_train = train_df.iloc[trn_idx][use_cols]
    y_train = Y.iloc[trn_idx]
    x_valid = train_df.iloc[val_idx][use_cols]
    y_valid = Y.iloc[val_idx]

    score, oof_pred, test_pred, feim, _, params = Classifier(
        model_type=model_type,
        x_train=x_train,
        y_train=y_train,
        x_valid=x_valid,
        y_valid=y_valid,
        x_test=[],
        params=params,
    )
    score_list.append(score)
    y_pred[val_idx] = oof_pred
    test_preds.append(test_pred)
    
    feim.rename(columns={'importance': f'importance_fold{n_fold+1}'}, inplace=True)
    feim.set_index('feature', inplace=True)
    feim_list.append(feim)
    
cv_score = np.mean(score_list)
feim_df = pd.concat(feim_list, axis=1)
feim_df['importance_avg'] = feim_df.mean(axis=1)

Training until validation scores don't improve for 100 rounds.
[200]	valid_0's auc: 0.88617
[400]	valid_0's auc: 0.887233
Early stopping, best iteration is:
[308]	valid_0's auc: 0.890163


SystemExit: 

In [ ]:
feim_df.sort_values(by='importance_avg', ascending=False, inplace=True)
display(feim_df)

In [70]:
COLUMNS_TOP100 = feim_df['importance_avg'].head(100).index
COLUMNS_TOP100

Index(['V258', 'C14', 'TransactionDT', 'C13', 'C1', 'C12', 'V149', 'card2',
       'C4', 'V317', 'card1', 'C8', 'TransactionAmt', 'V201', 'C11', 'D1',
       'D15', 'addr1', 'V189', 'D8', 'D2', 'V45', 'card5', 'V283', 'C10',
       'V187', 'D10', 'card3', 'dist1', 'D4', 'V34', 'D12', 'V207', 'C6',
       'V90', 'V294', 'V295', 'V312', 'V102', 'V313', 'V310', 'V69', 'D3',
       'V91', 'C9', 'V308', 'C2', 'V307', 'V323', 'V315', 'V82', 'C5', 'V87',
       'D11', 'V62', 'V44', 'V54', 'V209', 'V70', 'V257', 'V156', 'V320',
       'V165', 'V133', 'V94', 'V33', 'D14', 'V83', 'V74', 'V48', 'dist2',
       'V282', 'V53', 'V76', 'D13', 'V67', 'V318', 'D5', 'V131', 'V281',
       'V285', 'V49', 'V29', 'V266', 'V314', 'V36', 'V56', 'V61', 'V243', 'D6',
       'V130', 'D9', 'V12', 'addr2', 'V13', 'V30', 'V20', 'V66', 'V225',
       'V322'],
      dtype='object', name='feature')

In [2]:
HOME = os.path.expanduser('~')
gcp_config_path = f'{HOME}/privacy/gcp.yaml'
with open(gcp_config_path, 'r') as f:
    gcp_config = yaml.load(f)
    
is_create = False
dataset_name = 'dim_ml_dataset'
credentials  =  HOME + '/privacy/' + gcp_config['gcp_credentials']

# Delete
bq = BigQuery(credentials, dataset_name)

2019-07-16 22:44:02,782 func.BigQuery 31 [INFO]    [logger_func] start 
2019-07-16 22:44:03,011 func.BigQuery 49 [INFO]    [_set_dataset] Setup Dataset dim_ml_dataset. 


In [9]:
from bq_log import create_train_log_table, save_train_log

create_train_log_table(bq)

log_map = {}
log_map['exp_date']   = start_time
log_map['n_features'] = train_df.shape[1]
log_map['n_rows']     = train_df.shape[0]
log_map['cv_score']   = cv_score
log_map['metric']     = metric
log_map['model_type'] = model_type
save_train_log(log_map)